In [163]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import skrf as rf           # install with "pip install scikit-rf"

## 1. RCS of an aircraft wing

### a. Simulation

In [312]:
data = loadmat("data_wing_Shape_2023/data_wing_shape.mat")
data_wing_shape = data["data_wing_shape"]

x_pos = data_wing_shape[:,0]*1e-3               # x coordinate of each point [m]
y_pos = data_wing_shape[:,1]*1e-3               # y coordinate of each point  [m]
phi = data_wing_shape[:,2]                      # angle between the x-direction and the normal to the surface [rad]
curv_radius = data_wing_shape[:,3]*1e-3         # radius of curvature [m]

D = 2.7                                         # distance between wing and antennas [m]
N = len(x_pos)
sigma_simu = np.zeros(N)

theta_simu = np.zeros(N)

for i in range(N):
    r = curv_radius[i]
    x = x_pos[i]
    y = y_pos[i]
    if (y>=0):
       theta_simu[i] = np.arccos(x/(np.sqrt(x**2+y**2)))
    else:
        theta_simu[i] = -np.arccos(x/(np.sqrt(x**2+y**2))) + 2* np.pi
    sigma_simu[i] = np.pi * r * D


sorted_indices = np.argsort(theta_simu)
theta_simu = theta_simu[sorted_indices]*180/np.pi
sigma_simu = sigma_simu[sorted_indices]


plt.figure()
plt.grid()
plt.plot(theta_simu, sigma_simu)
plt.xlabel(r"$\theta$ [°]")
plt.ylabel(r"RCS $\sigma$ [m²]")
plt.show()

# HeatMap
x_min, x_max = np.min(x_pos), np.max(x_pos)
y_min, y_max = np.min(y_pos), np.max(y_pos)
x_grid, y_grid = np.linspace(x_min, x_max, len(x_pos)//10), np.linspace(y_min, y_max, len(y_pos)//10)
X, Y = np.meshgrid(x_grid, y_grid)

Z = np.zeros_like(X)
CurvRadius = np.zeros_like(Z)
for i in range(N):
    x_idx = np.argmin(np.abs(x_grid - x_pos[i]))
    y_idx = np.argmin(np.abs(y_grid - y_pos[i]))
    CurvRadius[y_idx, x_idx] = curv_radius[i]

plt.figure(figsize=(8, 6))
plt.pcolormesh(X, Y, np.log10(CurvRadius), cmap = "viridis",shading='auto')
plt.colorbar()
plt.grid()
plt.xlabel('X Position (m)')
plt.ylabel('Y Position (m)')
plt.title(r'Heatmap Curvature Radius')
plt.axis("equal")
plt.show()



C:\Users\mathi\AppData\Local\Temp\ipykernel_22136\2488426439.py:52: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(X, Y, np.log10(CurvRadius), cmap = "viridis",shading='auto')


### b. Comparison with measurements

In [282]:
# How to read s2p file

def extract_s2p(filename):
    snp = rf.Network(filename)
    # Network.f is the frequencies at which the S-parameter is evaluated in the snp file
    f = snp.f
    # Network.s is a matrix with the shape [len(f),n,n]
    S_param = snp.s
    S11mes = S_param[:,0,0]; S12mes = S_param[:,0,1]; S21mes = S_param[:,1,0]; S22mes = S_param[:,1,1]
    return f, S11mes, S12mes, S21mes,S22mes

def nat2dB(S):
    ampl = 20*np.log10(np.abs(S))   # [dB]
    phase = np.angle(S)             # [deg]
    return ampl,phase

def dB2natAmpl(S):
    ampl = 10**(S/20)
    return ampl


In [342]:
# Retrieve wing data

f, _, _, _ , _ = extract_s2p("data_lab/wing0.s2p")

N = 41      # number of measurements (angles)
step = 10   # step in degrees between 2 points

def S12_wing():
    # Return |S12| of the wing for all frequncies and angles
    S12_arr = np.zeros((len(f), N))    # (freq, angle))
    for i, angl in enumerate(range(0,N*step,step)):
        _ , _, S12, _ , _ = extract_s2p("data_lab/wing{}.s2p".format(angl))
        S12_arr[:,i] = np.abs(S12)
    return S12_arr

S12_mes = S12_wing()                # Matrix [freq, angle]
theta_mes = np.linspace(0,360,N)

# Retrieve RCS 
D = 2.7                     # [m]
Gr = 10**((15+21)/10)     # Gain of the receiving antenna
Gt = 10**(15/10)            # Gain of the transmitting antenna


lambd = 3e8/f               # [m] lambda = c/f, vector 

sigma_mes = np.zeros((len(f),N))
for i in range(len(f)):
    sigma_mes[i] = ((S12_mes[i]**(2))*(4*np.pi)**3 * (D**4)) / (Gr*Gt*(lambd[i]**2))


sigma_mes_mean = np.mean(sigma_mes, axis=0)
S12_mes_mean = np.mean(S12_mes, axis=1)

    
plt.figure()
plt.grid()
plt.plot(theta_mes, np.flip(sigma_mes_mean), label = "Measurements")
plt.plot(theta_simu, sigma_simu, label="Simulation")
plt.xlabel(r"$\theta$ [°]")
plt.ylabel(r"RCS $\sigma$ [m²]")
plt.legend()
#plt.savefig("comparison_meas_sim.pdf")
plt.show()



sigma_mes_mean_f = np.mean(sigma_mes, axis=1)

plt.figure()
plt.grid()
plt.plot(f/1e9, sigma_mes[:,0] ,label = "Measurements")
plt.xlabel(r"f [GHz]")
plt.ylabel(r"RCS $\sigma$ [m²]")
plt.legend()
#plt.savefig("RCS_f_0_degree.pdf")
plt.show()


plt.figure()
plt.grid()
plt.plot(f/1e9, nat2dB(S12_mes_mean)[0] ,label = "Measurements")
plt.xlabel(r"f [GHz]")
plt.ylabel(r"S21 [dB]")
plt.legend()
#plt.savefig("RCS_f_0_degree.pdf")
plt.show()



In [344]:
# RCS of the sphere

# Retrieve RCS 
D = 3.06 # [m]
Gr = 10**((15+21)/10)
Gt = 10**(15/10)


f_sphere, _, S12_sphere,_, _ = extract_s2p("data_lab/sphere.s2p")
S12_sphere = np.abs(S12_sphere)
lambd_sphere = 3e8/f_sphere
sigma_mes_sphere = (S12_sphere**2)*(4*np.pi)**3 * D**4 / (Gr*Gt*lambd_sphere**2)


R_sphere = 0.55/2
sigma_sphere_th = np.pi*(R_sphere**2)

plt.figure()
plt.grid()
plt.plot(f_sphere/1e9, sigma_mes_sphere ,label = "Measurements")
plt.plot(f_sphere/1e9, np.ones(len(f_sphere))*sigma_sphere_th ,label = "Theoretical")
plt.xlabel(r"f [GHz]")
plt.ylabel(r"RCS $\sigma$ [m²]")
plt.legend()
plt.show()




In [339]:

f_sphere, _, S12_sphere,_, _ = extract_s2p("data_lab/sphereReflection2.s2p")
S12_sphere = np.abs(S12_sphere)
lambd_sphere = 3e8/f_sphere
sigma_mes_sphere = (S12_sphere**2)*(4*np.pi)**3 * D**4 / (Gr*Gt*lambd_sphere**2)

plt.figure()
plt.grid()
plt.plot(f_sphere/1e9, nat2dB(S12_sphere)[0],label = "Measurements")

plt.xlabel(r"f [GHz]")
plt.ylabel(r"S12 [dB]")
plt.legend()
plt.show()